In [3]:
import os
import json
import time
import requests
import pandas as pd
import re 
from pathlib import Path
from tqdm import tqdm

# === API setup ===
api_key = "API key"
api_url = "https://api.anthropic.com/v1/messages"
model_name = "claude-3-haiku-20240307"

headers = {
    "x-api-key": api_key,
    "anthropic-version": "2023-06-01",
    "content-type": "application/json",
}

# === Paths ===
base_path = Path.home() / "Desktop" / "processed_subs"
subs_dir = base_path
summaries_dir = base_path / "summaries"
prompts_dir = base_path / "prompts_institutional_trust"
output_dir = base_path / "scored_institutional_trust"

prompts_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)

# === Load updated match file ===
matches_df = pd.read_csv("final_matches_updated2.csv")
matches_df = matches_df.dropna(subset=["subtitle_filename"])

# === API call ===

def call_claude(prompt):
    payload = {
        "model": model_name,
        "max_tokens": 400,
        "temperature": 0,
        "system": "You are a strict JSON-returning film analyst. Only respond in the precise JSON structure requested.",
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(api_url, headers=headers, json=payload)

    if response.status_code == 429:
        print("⚠️ Rate limit hit. Waiting 60 seconds...")
        time.sleep(60)
        return call_claude(prompt)

    if response.status_code != 200:
        print(f"⚠️ API Error {response.status_code}: {response.text}")
        return None

    data = response.json()
    content = data["content"][0]["text"]

    # 🛠 Fix common JSON colon error: change "score: 0.5" to "score": 0.5
    content = re.sub(r'"(score|confidence):', r'"\1":', content)

    try:
        return json.loads(content)
    except json.JSONDecodeError:
        print(f"⚠️ JSON parsing error:\n{content}")
        return None


# === Validation ===
EXPECTED_KEYS = [
    "formal_institutions_incl_military",
    "formal_institutions_excl_military",
    "military_institutions",
    "informal_cultural_norms",
    "informal_relationships",
    "informal_overall"
]

def is_valid_output(data):
    if not isinstance(data, dict):
        return False
    for key in EXPECTED_KEYS:
        if key not in data:
            return False
        val = data[key]
        if not isinstance(val, dict):
            return False
        if "score" not in val or "confidence" not in val:
            return False
        try:
            score = float(val["score"])
            conf = float(val["confidence"])
        except (ValueError, TypeError):
            return False
        if not (0.0 <= score <= 1.0) or not (0.0 <= conf <= 1.0):
            return False
    return True

# === Prompt generator ===
def create_institutional_trust_prompt(summary, dialogue_text):
    return f'''
You are analyzing how a film portrays societal trust in institutions. Use only the provided movie subtitles and/or Wikipedia plot summary.

We follow definitions from Douglass North (1990):
- Formal institutions: codified structures such as government, police, courts, military, and corporations.
- Informal institutions: unwritten norms, cultural values, family, friendships, and community relationships.

Your task is to return a JSON object with six trust scores and corresponding confidence scores, each ranging from 0 (very negative portrayal) to 1 (very positive portrayal). If a category is not meaningfully present in the film, assign a neutral score of 0.5 and a confidence ≤ 0.6.

⚠️ Do not include any explanation or extra text. Return only the JSON in the format provided below.

---
Please rate the following dimensions:

1. Formal Institutions (including military)
   - One overall score reflecting how trustworthy or competent formal institutions are portrayed, including the military.  
   - Consider government, law enforcement, courts, the military, and corporations.  

2. Formal Institutions (excluding military) 
   - Same as above, but **do not consider the military** in this score.  
   - Focus only on government, police, courts, and corporations.  

3. Military Institutions Only  
   - How is the military or national security apparatus portrayed? Are they heroic, competent, and respected—or corrupt, violent, or distrusted?  

4. Informal Institutions – Cultural Norms and Values
   - Are unwritten societal norms (e.g., justice, honesty, fairness, tradition, duty, rebellion) depicted positively or negatively?  
   - Is there a sense that these norms help society function, or are they broken or harmful?  

5. Informal Institutions – Interpersonal Relationships  
   - Are family, friendships, and community relationships portrayed as strong and trustworthy, or weak and broken?  
   - Do people rely on each other in meaningful ways?  

6. Overall Trust in Informal Institutions  
   - A general trust score reflecting how much informal institutions (cultural norms + relationships) appear to support stability, cooperation, or moral behavior in the film.

---


✅ Output format (strictly):

{{
  "formal_institutions_incl_military": {{"score": 0.0, "confidence": 0.0}},
  "formal_institutions_excl_military": {{"score": 0.0, "confidence": 0.0}},
  "military_institutions": {{"score": 0.0, "confidence": 0.0}},
  "informal_cultural_norms": {{"score": 0.0, "confidence": 0.0}},
  "informal_relationships": {{"score": 0.0, "confidence": 0.0}},
  "informal_overall": {{"score": 0.0, "confidence": 0.0}}
}}

Movie Summary:
\"\"\"{summary}\"\"\"

Dialogue Chunk:
\"\"\"{dialogue_text}\"\"\"
'''

# === Subtitle chunking ===
def chunk_dialogue(subs, chunk_size=5000, overlap=200):
    text_blocks = [line["text"].strip() for line in subs if line.get("text")]
    full_text = " ".join(text_blocks)

    chunks = []
    start = 0
    while start < len(full_text):
        end = start + chunk_size
        chunks.append(full_text[start:end])
        start = end - overlap
    return chunks

# === Main loop ===
for _, row in tqdm(matches_df.iterrows(), total=len(matches_df), desc="Scoring institutional trust (chunked)"):
    filename = row["subtitle_filename"]

    json_path = subs_dir / f"{filename}.json"
    summary_path = summaries_dir / f"{filename}_summary.txt"

    if not json_path.exists() or not summary_path.exists():
        print(f"⚠️ Missing files for {filename}. Skipping.")
        continue

    with open(json_path, "r", encoding="utf-8") as f:
        subs = json.load(f)

    with open(summary_path, "r", encoding="utf-8") as f:
        summary = f.read().strip()

    chunks = chunk_dialogue(subs, chunk_size=5000, overlap=200)

    for i, chunk_text in enumerate(chunks):
        output_path = output_dir / f"{filename}_chunk{i+1}_institutional_trust.json"

        if output_path.exists():
            print(f"⏩ Skipping already scored: {filename} chunk {i+1}")
            continue

        prompt = create_institutional_trust_prompt(summary, chunk_text)
        prompt_path = prompts_dir / f"{filename}_chunk{i+1}_institutional_trust_prompt.json"

        with open(prompt_path, "w", encoding="utf-8") as f:
            json.dump({"prompt": prompt}, f, indent=2)

        result = call_claude(prompt)

        if result is not None and is_valid_output(result):
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(result, f, indent=2)
            print(f"✅ Scored: {filename} chunk {i+1}")
        else:
            print(f"❌ Invalid or failed response: {filename} chunk {i+1}")

        time.sleep(1.5)


Scoring institutional trust (chunked):   0%|            | 0/478 [00:00<?, ?it/s]

⏩ Skipping already scored: 2017_Logan chunk 1
⏩ Skipping already scored: 2017_Logan chunk 2
⏩ Skipping already scored: 2017_Logan chunk 3
⏩ Skipping already scored: 2017_Logan chunk 4
⏩ Skipping already scored: 2017_Logan chunk 5
⏩ Skipping already scored: 2017_Logan chunk 6
⏩ Skipping already scored: 2017_Logan chunk 7
⏩ Skipping already scored: 2017_Logan chunk 8
⏩ Skipping already scored: 2017_Logan chunk 9
✅ Scored: 1979_10 chunk 1
⏩ Skipping already scored: 1979_10 chunk 2
⏩ Skipping already scored: 1979_10 chunk 3
⏩ Skipping already scored: 1979_10 chunk 4
⏩ Skipping already scored: 1979_10 chunk 5
⏩ Skipping already scored: 1979_10 chunk 6
⏩ Skipping already scored: 1979_10 chunk 7
⏩ Skipping already scored: 1979_10 chunk 8
✅ Scored: 1979_10 chunk 9


Scoring institutional trust (chunked):   0%|    | 2/478 [00:07<27:52,  3.51s/it]

✅ Scored: 1983_Tootsie chunk 1
⏩ Skipping already scored: 1983_Tootsie chunk 2
✅ Scored: 1983_Tootsie chunk 3
⏩ Skipping already scored: 1983_Tootsie chunk 4
⏩ Skipping already scored: 1983_Tootsie chunk 5
⏩ Skipping already scored: 1983_Tootsie chunk 6
✅ Scored: 1983_Tootsie chunk 7
⏩ Skipping already scored: 1983_Tootsie chunk 8
⏩ Skipping already scored: 1983_Tootsie chunk 9
✅ Scored: 1983_Tootsie chunk 10
✅ Scored: 1983_Tootsie chunk 11
✅ Scored: 1983_Tootsie chunk 12
⏩ Skipping already scored: 1983_Tootsie chunk 13
✅ Scored: 1983_Tootsie chunk 14


Scoring institutional trust (chunked):   1%|  | 3/478 [00:37<1:55:14, 14.56s/it]

...

Scoring institutional trust (chunked): 100%|▉| 476/478 [4:46:53<01:25, 42.67s/it

✅ Scored: 2025_Sinners chunk 1
✅ Scored: 2025_Sinners chunk 2
✅ Scored: 2025_Sinners chunk 3
✅ Scored: 2025_Sinners chunk 4
✅ Scored: 2025_Sinners chunk 5
✅ Scored: 2025_Sinners chunk 6
✅ Scored: 2025_Sinners chunk 7
✅ Scored: 2025_Sinners chunk 8
✅ Scored: 2025_Sinners chunk 9
✅ Scored: 2025_Sinners chunk 10
✅ Scored: 2025_Sinners chunk 11
✅ Scored: 2025_Sinners chunk 12
✅ Scored: 2025_Sinners chunk 13


Scoring institutional trust (chunked): 100%|▉| 477/478 [4:47:36<00:42, 42.89s/it

✅ Scored: 2025_Snow.White chunk 1
✅ Scored: 2025_Snow.White chunk 2
✅ Scored: 2025_Snow.White chunk 3
✅ Scored: 2025_Snow.White chunk 4
✅ Scored: 2025_Snow.White chunk 5
✅ Scored: 2025_Snow.White chunk 6
✅ Scored: 2025_Snow.White chunk 7
✅ Scored: 2025_Snow.White chunk 8
✅ Scored: 2025_Snow.White chunk 9


Scoring institutional trust (chunked): 100%|█| 478/478 [4:48:08<00:00, 36.17s/it


In [5]:
import json
import re
import pandas as pd
from pathlib import Path

# === Paths ===
input_dir = Path.home() / "Desktop" / "processed_subs" / "scored_institutional_trust"
per_film_csv_all_years = Path.home() / "Desktop" / "processed_subs" / "institutional_trust_per_film_scores_all_years.csv"
yearly_csv_all_years = Path.home() / "Desktop" / "processed_subs" / "institutional_trust_yearly_averages_all_years.csv"

# === Helper: Extract year from filename ===
def extract_year_from_filename(filename):
    match = re.search(r"(19|20)\d{2}", filename)
    return int(match.group(0)) if match else None

# === Aggregate per film across chunks ===
chunk_data = {}
for file in input_dir.glob("*_institutional_trust.json"):
    year = extract_year_from_filename(file.name)
    if year is None:
        print(f"⚠️ Skipping file with no year: {file.name}")
        continue

    film_id = "_".join(file.name.split("_")[:2])
    with open(file, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)

            if film_id not in chunk_data:
                chunk_data[film_id] = {
                    "year": year,
                    "scores": {
                        "formal_institutions_incl_military": [],
                        "formal_institutions_excl_military": [],
                        "military_institutions": [],
                        "informal_cultural_norms": [],
                        "informal_relationships": [],
                        "informal_overall": []
                    },
                    "confidences": {
                        "formal_institutions_incl_military": [],
                        "formal_institutions_excl_military": [],
                        "military_institutions": [],
                        "informal_cultural_norms": [],
                        "informal_relationships": [],
                        "informal_overall": []
                    }
                }

            for key in chunk_data[film_id]["scores"]:
                chunk_data[film_id]["scores"][key].append(data[key]["score"])
                chunk_data[film_id]["confidences"][key].append(data[key]["confidence"])

        except (json.JSONDecodeError, KeyError) as e:
            print(f"❌ Failed parsing {file.name}: {e}")
            continue

# === Create per-film DataFrame ===
film_records = []
for film_id, values in chunk_data.items():
    record = {
        "subtitle_filename": film_id,
        "year": values["year"]
    }

    for key in values["scores"]:
        scores = values["scores"][key]
        confidences = values["confidences"][key]
        record[key] = round(sum(scores) / len(scores), 3) if scores else None
        record[f"{key}_confidence"] = round(sum(confidences) / len(confidences), 3) if confidences else None

    film_records.append(record)

df_film = pd.DataFrame(film_records)
df_film.to_csv(per_film_csv_all_years, index=False)
print(f"✅ Saved per-film scores to: {per_film_csv_all_years}")

# === Compute yearly averages ===
if "year" in df_film.columns:
    df_yearly = df_film.groupby("year").mean(numeric_only=True).reset_index()
    df_yearly.to_csv(yearly_csv_all_years, index=False)
    print(f"✅ Saved yearly averages to: {yearly_csv_all_years}")
else:
    print("❌ No valid year column found — check your filenames and directory.")


✅ Saved per-film scores to: /Users/cedricroetheli/Desktop/processed_subs/institutional_trust_per_film_scores_all_years.csv
✅ Saved yearly averages to: /Users/cedricroetheli/Desktop/processed_subs/institutional_trust_yearly_averages_all_years.csv


In [7]:
import json
import re
import pandas as pd
from pathlib import Path

# === Paths ===
input_dir = Path.home() / "Desktop" / "processed_subs" / "scored_institutional_trust"
per_film_csv_weighted = Path.home() / "Desktop" / "processed_subs" / "institutional_trust_per_film_scores_weighted.csv"
yearly_csv_weighted = Path.home() / "Desktop" / "processed_subs" / "institutional_trust_yearly_averages_weighted.csv"

# === Helper: Extract year from filename ===
def extract_year_from_filename(filename):
    match = re.search(r"(19|20)\d{2}", filename)
    return int(match.group(0)) if match else None

# === Collect chunked scores by film ===
chunk_data = {}
for file in input_dir.glob("*_institutional_trust.json"):
    year = extract_year_from_filename(file.name)
    if year is None:
        print(f"⚠️ Skipping file with no year: {file.name}")
        continue

    film_id = "_".join(file.name.split("_")[:2])
    with open(file, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            if film_id not in chunk_data:
                chunk_data[film_id] = {
                    "year": year,
                    "scores": {
                        "formal_institutions_incl_military": [],
                        "formal_institutions_excl_military": [],
                        "military_institutions": [],
                        "informal_cultural_norms": [],
                        "informal_relationships": [],
                        "informal_overall": []
                    },
                    "confidences": {
                        "formal_institutions_incl_military": [],
                        "formal_institutions_excl_military": [],
                        "military_institutions": [],
                        "informal_cultural_norms": [],
                        "informal_relationships": [],
                        "informal_overall": []
                    }
                }

            for key in chunk_data[film_id]["scores"]:
                chunk_data[film_id]["scores"][key].append(data[key]["score"])
                chunk_data[film_id]["confidences"][key].append(data[key]["confidence"])

        except (json.JSONDecodeError, KeyError) as e:
            print(f"❌ Failed parsing {file.name}: {e}")
            continue

# === Compute weighted averages per film ===
film_records = []
for film_id, content in chunk_data.items():
    record = {
        "subtitle_filename": film_id,
        "year": content["year"]
    }

    for key in content["scores"]:
        scores = content["scores"][key]
        confidences = content["confidences"][key]
        weight_sum = sum(confidences)

        if scores and confidences and weight_sum > 0:
            weighted_avg = sum(s * c for s, c in zip(scores, confidences)) / weight_sum
            record[key] = round(weighted_avg, 3)
        else:
            record[key] = None  # fallback if no data or all confidences are zero

        # Optionally store unweighted average confidence
        if confidences:
            record[f"{key}_confidence"] = round(sum(confidences) / len(confidences), 3)
        else:
            record[f"{key}_confidence"] = None

    film_records.append(record)

# === Save weighted per-film scores
df_film_weighted = pd.DataFrame(film_records)
df_film_weighted.to_csv(per_film_csv_weighted, index=False)
print(f"✅ Saved weighted per-film scores to: {per_film_csv_weighted}")

# === Save weighted yearly averages
if "year" in df_film_weighted.columns:
    df_yearly_weighted = df_film_weighted.groupby("year").mean(numeric_only=True).reset_index()
    df_yearly_weighted.to_csv(yearly_csv_weighted, index=False)
    print(f"✅ Saved weighted yearly averages to: {yearly_csv_weighted}")
else:
    print("❌ No valid year column found — check your filenames and directory.")


✅ Saved weighted per-film scores to: /Users/cedricroetheli/Desktop/processed_subs/institutional_trust_per_film_scores_weighted.csv
✅ Saved weighted yearly averages to: /Users/cedricroetheli/Desktop/processed_subs/institutional_trust_yearly_averages_weighted.csv


In [11]:
import pandas as pd
from pathlib import Path

# === Path to previously stored per-film weighted scores ===
input_path = Path.home() / "Desktop" / "processed_subs" / "institutional_trust_per_film_scores_weighted.csv"

# === Load data ===
df = pd.read_csv(input_path)

# === Columns to include (excluding confidence columns) ===
trust_columns = [
    "formal_institutions_incl_military",
    "formal_institutions_excl_military",
    "military_institutions",
    "informal_cultural_norms",
    "informal_relationships",
    "informal_overall"
]

# === Calculate and print variances ===
print("📊 Variance of Per-Film Trust Scores:")
for col in trust_columns:
    var = df[col].var()
    print(f"- {col}: {var:.6f}")


📊 Variance of Per-Film Trust Scores:
- formal_institutions_incl_military: 0.011571
- formal_institutions_excl_military: 0.018556
- military_institutions: 0.005501
- informal_cultural_norms: 0.008013
- informal_relationships: 0.012867
- informal_overall: 0.010871


In [17]:
import pandas as pd
from pathlib import Path

# === Base directory path ===
base_dir = Path.home() / "Desktop" / "processed_subs"

# === Filenames for all four datasets ===
files = {
    "Per-Film (Unweighted)": "institutional_trust_per_film_scores_all_years.csv",
    "Per-Film (Weighted)": "institutional_trust_per_film_scores_weighted.csv",
    "Yearly (Unweighted)": "institutional_trust_yearly_averages_all_years.csv",
    "Yearly (Weighted)": "institutional_trust_yearly_averages_weighted.csv"
}

# === Columns to include (excluding confidence columns) ===
trust_columns = [
    "formal_institutions_incl_military",
    "formal_institutions_excl_military",
    "military_institutions",
    "informal_cultural_norms",
    "informal_relationships",
    "informal_overall"
]

# === Compute and display variances for each file ===
for label, filename in files.items():
    file_path = base_dir / filename

    try:
        df = pd.read_csv(file_path)
        print(f"\n Variance of Trust Scores – {label}:")
        for col in trust_columns:
            if col in df.columns:
                var = df[col].var()
                print(f"- {col}: {var:.6f}")
            else:
                print(f"- {col}: ⚠️ Not found in {filename}")
    except FileNotFoundError:
        print(f"\n❌ File not found: {filename}")
    except Exception as e:
        print(f"\n❌ Error reading {filename}: {e}")



 Variance of Trust Scores – Per-Film (Unweighted):
- formal_institutions_incl_military: 0.011432
- formal_institutions_excl_military: 0.018482
- military_institutions: 0.005072
- informal_cultural_norms: 0.007665
- informal_relationships: 0.012807
- informal_overall: 0.010586

 Variance of Trust Scores – Per-Film (Weighted):
- formal_institutions_incl_military: 0.011571
- formal_institutions_excl_military: 0.018556
- military_institutions: 0.005501
- informal_cultural_norms: 0.008013
- informal_relationships: 0.012867
- informal_overall: 0.010871

 Variance of Trust Scores – Yearly (Unweighted):
- formal_institutions_incl_military: 0.001133
- formal_institutions_excl_military: 0.001793
- military_institutions: 0.000525
- informal_cultural_norms: 0.000896
- informal_relationships: 0.001808
- informal_overall: 0.001311

 Variance of Trust Scores – Yearly (Weighted):
- formal_institutions_incl_military: 0.001147
- formal_institutions_excl_military: 0.001812
- military_institutions: 0.000

In [21]:
import pandas as pd
import numpy as np
from pathlib import Path

# === Paths ===
input_path = Path.home() / "Desktop" / "processed_subs" / "institutional_trust_per_film_scores_all_years.csv"
output_unweighted = Path.home() / "Desktop" / "processed_subs" / "yearly_averages_filtered_unweighted.csv"
output_weighted = Path.home() / "Desktop" / "processed_subs" / "yearly_averages_filtered_weighted.csv"

# === Load data ===
df = pd.read_csv(input_path)

# === Define score columns and their corresponding confidence columns ===
score_cols = [
    "formal_institutions_incl_military",
    "formal_institutions_excl_military",
    "military_institutions",
    "informal_cultural_norms",
    "informal_relationships",
    "informal_overall"
]

confidence_cols = [f"{col}_confidence" for col in score_cols]

# === Replace exact 0.5 scores with NaN ===
for col in score_cols:
    df[col] = df[col].apply(lambda x: np.nan if x == 0.5 else x)

# === UNWEIGHTED: Group by year and calculate mean (ignores NaNs) ===
df_unweighted = df.groupby("year")[score_cols].mean(numeric_only=True).reset_index()

# === WEIGHTED: Multiply each score by its confidence, then divide by sum of confidence ===
weighted_scores = {}

for col, conf_col in zip(score_cols, confidence_cols):
    product_name = f"{col}_weighted"
    df[product_name] = df[col] * df[conf_col]
    weighted_sum = df.groupby("year")[product_name].sum()
    confidence_sum = df.groupby("year")[conf_col].sum()
    weighted_scores[col] = weighted_sum / confidence_sum

# === Combine into one DataFrame ===
df_weighted = pd.DataFrame(weighted_scores).reset_index()

# === Save CSVs ===
df_unweighted.to_csv(output_unweighted, index=False)
df_weighted.to_csv(output_weighted, index=False)

print(f"✅ Saved unweighted averages to: {output_unweighted.name}")
print(f"✅ Saved weighted averages to: {output_weighted.name}")


✅ Saved unweighted averages to: yearly_averages_filtered_unweighted.csv
✅ Saved weighted averages to: yearly_averages_filtered_weighted.csv


In [25]:
import pandas as pd
from pathlib import Path
import numpy as np

# === File paths ===
input_path = Path.home() / "Desktop" / "processed_subs" / "institutional_trust_per_film_scores_all_years.csv"
output_path = Path.home() / "Desktop" / "processed_subs" / "institutional_trust_per_film_scores_filtered.csv"

# === Load data ===
df = pd.read_csv(input_path)

# === Trust score columns and corresponding confidence columns ===
score_cols = [
    "formal_institutions_incl_military",
    "formal_institutions_excl_military",
    "military_institutions",
    "informal_cultural_norms",
    "informal_relationships",
    "informal_overall"
]
confidence_cols = [f"{col}_confidence" for col in score_cols]

# === Filter: Replace scores of 0.5 with NaN if confidence ≤ 0.6 ===
for score_col, conf_col in zip(score_cols, confidence_cols):
    mask = (df[score_col] == 0.5) & (df[conf_col] <= 0.6)
    df.loc[mask, score_col] = np.nan

# === Save filtered dataset ===
df.to_csv(output_path, index=False)
print(f"✅ Filtered file saved to: {output_path.name}")

# === Calculate unweighted variance ===
print("\n Unweighted Variance (filtered):")
for col in score_cols:
    var = df[col].var()
    print(f"- {col}: {var:.6f}")

# === Calculate weighted variance ===
print("\n Weighted Variance (filtered):")
for col, conf_col in zip(score_cols, confidence_cols):
    x = df[col]
    w = df[conf_col]
    mask = (~x.isna()) & (~w.isna())
    x = x[mask]
    w = w[mask]

    if len(x) > 1:
        weighted_mean = (x * w).sum() / w.sum()
        weighted_var = ((w * (x - weighted_mean) ** 2).sum()) / w.sum()
        print(f"- {col}: {weighted_var:.6f}")
    else:
        print(f"- {col}: insufficient data")


✅ Filtered file saved to: institutional_trust_per_film_scores_filtered.csv

 Unweighted Variance (filtered):
- formal_institutions_incl_military: 0.011332
- formal_institutions_excl_military: 0.016654
- military_institutions: 0.006732
- informal_cultural_norms: 0.007856
- informal_relationships: 0.012673
- informal_overall: 0.010600

 Weighted Variance (filtered):
- formal_institutions_incl_military: 0.011655
- formal_institutions_excl_military: 0.016654
- military_institutions: 0.007727
- informal_cultural_norms: 0.008248
- informal_relationships: 0.012818
- informal_overall: 0.011012
